In [1]:
%%html
<style>
.widget-label {
    white-space: nowrap;
}
.container {
    width: 100%;
}
.p-Widget {
    -webkit-user-select: text;
}
</style>

In [83]:
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as wid
from bokeh.models import ColumnDataSource
from bokeh.layouts import widgetbox
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Panel, Tabs, NumberFormatter
from bokeh.io import output_notebook, show
import heron
import pandas as pd
import numpy as np
import sprockets
import os
import time
import gzip
import bokeh.models as bom
import bokeh.plotting as bop
from pandas.tseries.offsets import BDay

bop.output_notebook()

dates = ['pick a date','20170831']
max_interval = 10000

class Dash:
    def __init__(self):
        self.bird_displayed = False
        self.trades_loaded = False
        self.wids_displayed = False
        self.num_days = 1
        self.desks = []
        self.interval = 480
        
        self.bb_log_folder = "/net/audits/Blackbird/"
        
        output_notebook()

        self.init_date_picker()
        self.init_num_days_slider()
        self.init_interval_slider()
        self.init_desk_picker()
        
        self.bird_button = wid.Button(description="Load Birds")
        self.bird_button.on_click(self.on_bird_button_clicked)
        self.bird_button.button_style = 'primary'
        
        self.button = wid.Button(description="Find matching trades")
        self.button.on_click(self.on_button_clicked)
        self.button.button_style = 'primary'

    def on_button_clicked(self,b):
        clear_output()
        self.prog = wid.FloatProgress(min=0, max=100)
        self.get_matching_trades()
        self.format_matching_trades()
        self.show_fig()
        self.show_summary()
        self.show_table()
        
    def on_bird_button_clicked(self,b):
        self.init_bird_picker()
        self.reveal_bird_wid()

    def on_date(self,x):
        clear_output()
        self.date = x.new.replace("-","")
        self.desks = []
        self.desks += heron.api.get_available_desks(self.date)
        self.desks.sort()
        self.desks.insert(0,'pick a desk')

        self.trades_loaded = False
        
        self.init_desk_picker()
        
        if self.bird_displayed:
            self.bird_picker.value = ()
            self.bird_picker.selected_labels = ()
        
    def on_desk(self,x):
        if x.new != 'pick a desk':
            clear_output()
            self.desk = x.new
            self.trades = pd.DataFrame()
            
            end_date = pd.to_datetime(self.date)
            start_date = end_date - np.timedelta64(self.num_days - 1,'D')
            for date in pd.date_range(start_date, end_date, freq='B', unit='ms'):
                self.trades = self.trades.append(heron.get_trades(str(date).replace("-","")[:8], self.desk))
                
            self.avail_birds = self.trades[self.trades.event == "Quote Trade"].bird.unique()
            self.trades_loaded = True

    def on_num_days(self,x):
        self.num_days = x.new
        self.trades_loaded = False
        
    def on_bird(self,x):
        self.birds = x.new
        
    def on_interval(self,x):
        clear_output()
        self.interval = x.new
        
    def init_desk_picker(self):
        if not self.wids_displayed:
            self.desk_picker = wid.Dropdown(
                options=self.desks,
                description='Desk:',
            )
            self.desk_picker.observe(self.on_desk,names='value')
        else:
            self.desk_picker.options = self.desks
            self.desk_picker.value = self.desks[0]
            
    def init_bird_picker(self):
        if not self.bird_displayed:
            self.bird_picker =  wid.SelectMultiple(
                options=self.avail_birds.tolist(),
                rows=len(self.avail_birds),
                layout=wid.Layout(height="200px"),
                description='Birds',
                disabled=False
            )
            self.bird_picker.observe(self.on_bird,names='value')
        else:
            self.bird_picker.value = ()
            self.bird_picker.selected_labels = ()
            self.bird_picker.options = self.avail_birds.tolist()

        
    def init_date_picker(self):
        self.date_picker = sprockets.DatePicker()
        self.date_picker.observe(self.on_date,names='value')
        today = pd.datetime.today()
        self.date_picker.value = str(today - BDay(1))[:10]
        
    def init_interval_slider(self):
        self.interval_slider = wid.IntSlider(
            value=self.interval,
            min=0,
            max=max_interval,
            step=10,
            description=' Offset (ms):    ',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
        )
        self.interval_slider.observe(self.on_interval,names='value')
        
    def init_num_days_slider(self):
        self.num_days_slider = wid.IntSlider(
            value=1,
            min=0,
            max=100,
            step=1,
            description='Num Days:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        self.num_days_slider.observe(self.on_num_days,names='value')
            
            
    def reveal_wids(self):
        if not self.wids_displayed:
            self.wid_box = wid.VBox([wid.HBox([self.date_picker,
                                   self.num_days_slider,
                                   self.interval_slider,
                                   self.desk_picker,
                                   self.bird_button])
                        ], width='100%')
            
            display(self.wid_box)
            self.wids_displayed = True
    
    def reveal_bird_wid(self):
        if not self.bird_displayed:
            self.bird_wid = wid.VBox([wid.HBox([self.bird_picker, self.button])], width='100%')
            display(self.bird_wid)
            self.bird_displayed = True
    
    def get_matching_trades(self):
        
        display(self.prog)
        prog_inc =  1 + 100 / ((self.num_days * len(self.birds)) + self.num_days)
        
        self.start_time = time.time()
        
        if not self.trades_loaded:
            self.trades = pd.DataFrame()
            end_date = pd.to_datetime(self.date)
            start_date = end_date - np.timedelta64(self.num_days - 1,'D')
            
            for date in pd.date_range(start_date, end_date, freq='B', unit='ms'):
                self.trades = self.trades.append(heron.get_trades(str(date).replace("-","")[:8], self.desk))
                self.prog.value += prog_inc/2
            
        self.trades["time"] = pd.to_datetime(self.trades["time"], unit="ns").dt.tz_localize("UTC").dt.tz_convert("US/Central").dt.tz_localize(None)
        self.trades = self.trades.sort_values(["time"])
        self.trades["reload_time"] = pd.to_datetime(0)
        self.trades["time_after_bulk_delete"] = pd.to_timedelta(0)
        
        self.after_load_trades = time.time()
        
        self.reload_times = pd.DataFrame()
        
        end_date = pd.to_datetime(self.date)
        start_date = end_date - np.timedelta64(self.num_days - 1,'D')

        for date in pd.date_range(start_date, end_date, freq='B', unit='ms'):
            date = str(date).replace("-","")[:8]
            for bird in self.birds:
                new_reloads = pd.DataFrame()
                try:
                    with open(self.bb_log_folder + date + "/" + "blackbird_" + bird + ".log") as infile:
                        new_times = pd.Series([line.split("][")[1] for line in infile if "QuoteManager::check_bulk_delete_reload" in line])
                        new_reloads['reload_time'] = new_times
                        new_reloads['bird'] = bird
                        if len(new_times) > 0:
                            self.reload_times = self.reload_times.append(new_reloads)
                except FileNotFoundError:
                    try:
                        with gzip.open(self.bb_log_folder + date + "/" + "blackbird_" + bird + ".gz") as infile:
                            new_times = pd.Series([line.split("][")[1] for line in infile if "QuoteManager::check_bulk_delete_reload" in line])
                            new_reloads['reload_time'] = new_times
                            new_reloads['bird'] = bird
                            if len(new_times) > 0:
                                self.reload_times = self.reload_times.append(new_reloads)
                    except: 
                        pass
            self.prog.value += prog_inc
        
        self.reload_times['reload_time'] = pd.to_datetime(self.reload_times['reload_time'])
        self.reload_times = self.reload_times.sort_values(['reload_time'])
            
            
        self.after_load_reload_times = time.time()
        
        time_delta = np.timedelta64(self.interval, 'ms')
        self.matching_trades = pd.DataFrame()
        
        i = 0
        for ix, reload_time in self.reload_times.iterrows():
            trade_time = self.trades.iloc[i]["time"]
            while trade_time <= reload_time['reload_time'] and i < len(self.trades) - 1:
                i += 1
                trade_time = self.trades.iloc[i]["time"]
            
            start = i # need to save start of interval for trades that fall within multiple reload intervals, we only keep the last one
            while trade_time > reload_time['reload_time'] and trade_time < (reload_time['reload_time'] + time_delta) and i < len(self.trades) - 1:
                if self.trades.iloc[i]["bird"] == reload_time['bird'] and self.trades.iloc[i]["event"] == "Quote Trade":
                    self.trades.iloc[i, self.trades.columns.get_loc('reload_time')] = reload_time['reload_time']
                    self.trades.iloc[i, self.trades.columns.get_loc('time_after_bulk_delete')] = self.trades.iloc[i]["time"] - reload_time['reload_time']
                    self.matching_trades = self.matching_trades.append(self.trades.iloc[i])
                i += 1
                trade_time = self.trades.iloc[i]["time"]
            i = start
            self.prog.value += prog_inc/len(self.reload_times)

        self.matching_trades = self.matching_trades.drop_duplicates(subset=['bird', 'time'], keep='last') 
        
        self.prog.close()
        
        self.after_matching_trades = time.time()
        
        
    def format_matching_trades(self):
        
        if len(self.matching_trades) > 0:
            self.matching_trades["time"] = self.matching_trades["time"].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S.%f"))
            self.matching_trades["reload_time"] = self.matching_trades["reload_time"].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S.%f"))
            
            self.matching_trades['cash_edge'] = self.matching_trades['edge'] * self.matching_trades['trade_qty'] * self.matching_trades['option_multiplier']
            self.matching_trades['cash_offset'] = self.matching_trades['offset'].abs() * self.matching_trades['trade_qty'] * self.matching_trades['option_multiplier']
            self.matching_trades['cash_dq']   = self.matching_trades['dq_100ms'] * self.matching_trades['delta'].abs() * self.matching_trades['trade_qty'] * self.matching_trades['option_multiplier']
            self.matching_trades['cash_vq']   = self.matching_trades['vq_60min'] * self.matching_trades['trade_qty'] * self.matching_trades['option_multiplier']
            
            
            self.matching_trades['cash_retained_edge_ex_vq'] = self.matching_trades['cash_edge'] + self.matching_trades['cash_dq']
            self.matching_trades['cash_retained_edge']       = self.matching_trades['cash_edge'] + self.matching_trades['cash_dq'] + self.matching_trades['cash_vq']
            
            self.matching_trades['retained_edge']       = self.matching_trades['cash_retained_edge'] / self.matching_trades['cash_offset']
            self.matching_trades['retained_edge_ex_vq'] = self.matching_trades['cash_retained_edge_ex_vq'] / self.matching_trades['cash_offset']
            

            self.matching_trades = self.matching_trades.sort_values(['time'])
            return self.matching_trades
        else:
            return pd.DataFrame()
        
    def show_summary(self):
        cash_formatter    = NumberFormatter()
        cash_formatter.format    = "$0,0.00"
        cash_formatter.format    = "$0,0.00"
        
        print(len(self.matching_trades)," matching trades")
        print('Total offset:              ${:0,.2f}'.format(round(self.matching_trades['cash_offset'].sum(),2)))
        print('Total retained edge ex-vq: ${:0,.2f}'.format(round(self.matching_trades['cash_retained_edge_ex_vq'].sum(),2)), " [", round(self.matching_trades['cash_retained_edge_ex_vq'].sum()/self.matching_trades['cash_offset'].sum()*100,2), "% ]")
        print('Total retained edge:       ${:0,.2f}'.format(round(self.matching_trades['cash_retained_edge'].sum(),2)), " [", round(self.matching_trades['cash_retained_edge'].sum()/self.matching_trades['cash_offset'].sum()*100,2), "% ]")
        
    def show_table(self):
        if len(self.matching_trades) > 0:
            data = dict(self.matching_trades[['reload_time','time','bird','time_after_bulk_delete','cash_offset','cash_edge','cash_dq','cash_vq','retained_edge_ex_vq', 'retained_edge']])
            cash_formatter    = NumberFormatter()
            percent_formatter = NumberFormatter()
            decimal_formatter = NumberFormatter()
            cash_formatter.format    = "$0,0.00"
            percent_formatter.format = '0%'
            decimal_formatter.format = '0.0'
            source = ColumnDataSource(data)
            columns = [
                TableColumn(field="time_after_bulk_delete", title="time after reload (ms)"),
                TableColumn(field="time", title="trade time"),
                TableColumn(field='cash_offset', title="cash offset", formatter=cash_formatter),
                TableColumn(field='cash_edge', title="cash edge", formatter=cash_formatter),
                TableColumn(field="cash_vq", title="cash vq", formatter=cash_formatter),
                TableColumn(field="cash_dq", title="cash dq", formatter=cash_formatter),
                TableColumn(field="retained_edge_ex_vq", title="retained edge ex-vq", formatter=percent_formatter),
                TableColumn(field="retained_edge", title="retained edge", formatter=percent_formatter),
                TableColumn(field="bird", title="bird"),
            ]
            data_table = DataTable(source=source, columns=columns, width=1000, height=280)

            show(widgetbox(data_table))
        
            
    def show_fig(self):
        if len(self.matching_trades) > 0:
            
            fig1 = bop.figure(width=900, height=600)
            fig1.scatter(self.matching_trades['time_after_bulk_delete'], self.matching_trades['retained_edge_ex_vq'], color="firebrick")
            fig1.yaxis[0].axis_label = "Retained Edge ex-vq"
            fig1.xaxis[0].axis_label = "Time after reload"
            
            tab1 = Panel(child=fig1, title="Retained Edge ex-vq")
            
            fig2 = bop.figure(width=900, height=600)
            fig2.scatter(self.matching_trades['time_after_bulk_delete'], self.matching_trades['retained_edge'], color="firebrick")
            fig2.yaxis[0].axis_label = "Retained Edge"
            fig2.xaxis[0].axis_label = "Time after reload"
            
            tab2 = Panel(child=fig2, title="Retained Edge")
            
            self.tabs = Tabs(tabs=[ tab1, tab2 ])
            bop.show(self.tabs)
        
#         print("load trades time: ", self.after_load_trades - self.start_time)
#         print("load reloads time: ", self.after_load_reload_times - self.after_load_trades)
#         print("find matching trades time: ", self.after_matching_trades - self.after_load_reload_times)
#         print("display figs time: ", time.time() - self.after_matching_trades)

Loading BokehJS ...

In [84]:
dash = Dash()
dash.reveal_wids()